In [ ]:
##最新可用版本
import random
import sympy
import secrets
import gmpy2
import re
import numpy as np
import time
import hashlib
import os
import pickle

# random.seed(6)
# np.random.seed(6)

# def alldct(num):
USER_ID = int(os.getenv('USER_ID', 1))
NUM_I = int(os.getenv('NUM_I', 1))
ROLE = os.getenv('ROLE')  # 'worker' or 'final'
SHARE_DIR = '/app/share'
# 创建共享目录
os.makedirs(SHARE_DIR, exist_ok=True)
def alldct(num, mode=mode, user_index=0):
    def sample_discrete_gaussian(mean, std, dim, c=0.5):  
        """  
        生成一个符合离散高斯分布的随机向量。  

        :param mean: 均值（可以是标量或向量，如果是向量，则每个元素分别对应一个维度的均值）  
        :param std: 标准差（可以是标量或向量）  
        :param dim: 向量的维度  
        :param c: 截断参数，用于四舍五入或取整前的阈值调整  
        :return: 符合离散高斯分布的随机向量  
        """  
        # 如果mean和std是标量，则扩展为向量  
        if np.isscalar(mean):  
            mean = np.full(dim, mean)  
        if np.isscalar(std):  
            std = np.full(dim, std)  

        # 从连续高斯分布中采样  
        samples = np.random.normal(loc=mean, scale=std, size=dim)  
        #print("samples",list(samples))
        # 根据截断参数进行四舍五入  
        discrete_samples = np.round(samples / c) * c  
        return discrete_samples
    def myrandvec(size,prime):
        return [random.randint(0, prime) for i in range(size)]
    def generate_polynomial(secret, threshold, prime):  
        """  
        生成一个随机多项式，其中常数项为secret，且多项式的阶数为threshold-1  
        """  
        #coeffs = [secrets.randbelow(int(prime)) for _ in range(threshold - 1)] + [secret] 
        coeffs =myrandvec(threshold - 1,prime)+[secret] ##测试用
        return sympy.Poly(coeffs, sympy.symbols('x'))  

    def share_secret(secret, threshold, num_shares, prime):  
        """  
        分享秘密到多个份额  
        """  
        poly = generate_polynomial(secret, threshold, prime)  
        #print("poly",poly)
        shares = []  
        for i in range(1, num_shares + 1):  
            y = (poly.subs(sympy.symbols('x'), i)) % prime
            shares.append((i, y))  
        return shares  

    def recover_secret(shares, prime):  
        """  
        从足够的份额中恢复秘密  
        """  
        if len(shares) < 2:  
            raise ValueError("至少需要两个份额来恢复秘密")  

        x_values = [share[0] for share in shares]  
        y_values = [share[1] for share in shares]  

        # 使用拉格朗日插值法来恢复多项式  
        terms = []  
        for i, (x_i, y_i) in enumerate(shares):  
            p = 1  
            for j, (x_j, _) in enumerate(shares):  
                if i != j: 
                    p *= (sympy.symbols('x') - x_j) *mod_inverse(x_i - x_j,prime)  
            terms.append(y_i * p)  

        poly = sympy.Add(*terms)  
        #print("poly",poly)
        secret = poly.subs(sympy.symbols('x'), 0) % prime  
        return int(secret)  
    def recover_secret_special(ys, xs,N_Factor,Delta,q):  
        """  
        从足够的份额中恢复秘密  
        """  
        if len(ys) < 2:  
            raise ValueError("至少需要两个份额来恢复秘密")  

        x_values = xs  
        y_values = ys 
        shares=list(zip(xs, ys))
        #print("shares",shares)
        # 使用拉格朗日插值法来恢复多项式  
        terms = []  
        for i, (x_i, y_i) in enumerate(shares):  
            p = 1  
            for j, (x_j, _) in enumerate(shares):  
                if i != j: 
                    p *= (sympy.symbols('x') - x_j) /(x_i - x_j)  
            terms.append(y_i * N_Factor * p)  

        poly = sympy.Add(*terms)  
        #print("poly",poly)
        secret = poly.subs(sympy.symbols('x'), 0)  
        return int(secret%q) 
    def mydot(l1,l2):
        sum=0
        for i in range(len(l1)):
            sum+=int(l1[i])*int(l2[i])
        return sum

    def mymod(n,p):
        if (n % p) <p//2:
            return n%p
        return n%p-p
    def myvecm(scal,vec):
        res=[]
        for i in range(len(vec)):
            res+=[int(int(vec[i])*scal)]
        return res
    def myveca(vecl,vecr):
        res=[]
        for i in range(len(vecl)):
            res+=[int(vecl[i])+int(vecr[i])]
        return res
    def myvecmod(q,vec):
        res=[]
        for i in range(len(vec)):
            res+=[int(vec[i])%q]
        return res
    def SamplePoly(d,q):
        coef=[random.randint(0, q-1) for i in range(d)]
        f=coef[d-1]*x**(d-1)
        for i in range(1,d):
            f+=coef[d-1-i]*x**(d-1-i)
        return f
    def SamplePolyCo(d,coef):
        #print("coef[d-1]",coef[d-1])
        coef=ChangeType(coef)
        f=coef[d-1]*x**(d-1)
        for i in range(1,d):
            f+=coef[d-1-i]*x**(d-1-i)
        return f
    def GetCoef(d,qq):
        strf="{}".format(qq)
        strfs=strf.split("+")
        coefs=[0]*d
        pattern=r'\d+'
        for item in strfs:
            #两个数字或者一个数字
            if "*" in item and "^" in item:
                sa,sb=re.findall(pattern,item)
                coefs[int(sb)]=int(sa)
            elif "^" in item:
                sb=re.findall(pattern,item)
                coefs[int(sb[0])]=1
            elif "*" in item:
                sa=re.findall(pattern,item)
                coefs[1]=int(sa[0])
            else:
                sa=re.findall(pattern,item)
                if sa!=[]:
                    coefs[0]=int(sa[0])##常数项
                else:
                    coefs[1]=1##一次项且系数为1
        return coefs
    def ChangeType(xa):
        res=[]
        for item in xa:
            res+=[int(item)]
        return res
    def o2f(vec):
        sum=0
        for it in vec:
            sum+=it*it
        return int(math.sqrt(sum))
    def oinf(vec):
        res=0
        for it in vec:
            v=abs(it)
            if v>res:
                res=v
        return res
    def CheckEqu(v1,v2):
        lv1=len(v1)
        lv2=len(v2)
        if lv1!=lv2:
            return false
        else:
            for i in range(lv1):
                if v1[i]!=v2[i]:
                    return false
        return true

    def SampleMatrix(n,m,d,q):
        matrix=[]
        for i in range(n):
            irow=[]
            for j in range(m):
                irow+=[SamplePoly(d,q)]
            matrix+=[irow]
        return matrix
    def SampleGas(d,m,s):
        res=[]
        for i in range(m):
            tmp=sample_discrete_gaussian(0, s, d, c=1)
            tmppoly=SamplePolyCo(d,tmp)
            res+=[tmppoly]
        return res
    def Hashf(stri,m,d,q):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(stri.encode('utf-8'))
        col=[]
        for i in range(m):
            row=[]
            for j in range(d):
                index=int(i*d+j)
                sha256_hash.update(index.to_bytes((index.bit_length() + 7) // 8, byteorder='big'))
                hash_hex=sha256_hash.hexdigest()
                val=int(hash_hex[:20], 16)
                row+=[val%q]
            col+=[SamplePolyCo(d,row)]
        return col
    def HashC(vecx,d,kappa):##LPN22##输入是多个多项式
        stri=""
        for item in vecx:
            for po in item:
                stri+="{}".format(po)
        sha256_hash = hashlib.sha256()
        sha256_hash.update(stri.encode('utf-8'))
        hd=d//2
        coef=[]
        for j in range(hd):
            sha256_hash.update(j.to_bytes((j.bit_length() + 7) // 8, byteorder='big'))
            hash_hex=sha256_hash.hexdigest()
            val=int(hash_hex[:8], 16)
            coef+=[val%kappa]
        coef=coef+[0]+[-i for i in coef[1:]][::-1]     
        f=coef[d-1]*x**(d-1)
        for i in range(1,d):
            f+=coef[d-1-i]*x**(d-1-i)
        return f###采用hashC时发现需要有负数存在,然后在sage表示中，用的是0,q。因此负数非常大
    def MatrixVectorMul(A,b,n,m,d,q):
        res=[SamplePolyCo(d,[0]*d) for i in range(n)]
        for i in range(n):
            for j in range(m):
                res[i]+=A[i][j]*b[j]
        return res
    def Vectoradd(a,b):
        ln=len(a)
        res=[]
        for i in range(ln):
            res+=[a[i]+b[i]]
        return res
    def Vectorsub(a,b):
        ln=len(a)
        res=[]
        for i in range(ln):
            res+=[a[i]-b[i]]
        return res
    def Vectormul(a,b):
        ln=len(a)
        res=SamplePolyCo(d,[0]*d)
        for i in range(ln):
            res+=a[i]*b[i]
        return res
    def VecInner(a,b,q):
        acoef=[]
        bcoef=[]
        for ai in a:
            acoef+=GetCoef(d,ai)
        for bi in b:
            bcoef+=GetCoef(d,bi)
        res=0
        ln=len(acoef)
        for i in range(ln):
            res+=mymod(acoef[i],q)*mymod(bcoef[i],q)
        return res 

    def Shareing(veck,kappa,d,N,T,q):
        veckUc=[[[0]*d for j in range(kappa)] for i in range(N)]
        i=0
        for item in veck:
            itemc=GetCoef(d,item)
            j=0
            for coef in itemc:
                tmp=share_secret(coef, T, N, q)
                for k in range(N):
                    #print("k, i, j ",k,i,j)
                    veckUc[k][i][j]=tmp[k][1]
                j=j+1
            i=i+1
        vecU=[[0 for j in range(kappa)] for i in range(N)]

        j=0
        for item in veckUc:
            k=0
            for itemm in item:


                vecU[j][k]=SamplePolyCo(d,itemm)

                k=k+1
            j=j+1
        return vecU
    def Reg1(vecz,vecv,s,M,q):
        u=random.random()
        zv=VecInner(vecz,vecv,q)
        #print(zv,type(zv))
        vv=VecInner(vecv,vecv,q)
        #print(vv,type(vv))
        #print(s,type(s))
        ei=(-2*zv+vv)/(2*s**2)
        #print("ei",ei)
        rv=1/M*math.e**ei
        #print(rv)
        if u>rv:
            return 1
        else:
            return 0

    def GenPi(bfA,bfB,hx,vecru,vecrup,kappa,m,d,stdev,s,Bc):
        pi=[]
        iter=10*int(M+1)
        for i in range(iter):
            vecy1=SampleGas(d,m,stdev)
            vecy2=SampleGas(d,m,stdev)
            vecw1=MatrixVectorMul(bfA,vecy1,n,m,d,q)
            vecw2=MatrixVectorMul(bfA,vecy2,n,m,d,q)
            vecv1=MatrixVectorMul(bfB,vecy1,kappa,m,d,q)
            vecv11=Vectormul(vecv1,hx)
            vecv2=vecy2[0]
            vecv=vecv11-vecv2
            c=HashC([vecw1,vecw2,[vecv]],d,Bc)
            # print("c:",c)
            v1=[c*i for i in vecru]
            z1=Vectoradd(v1,vecy1)
            v2=[c*i for i in vecrup]
            z2=Vectoradd(v2,vecy2)
            if Reg1(z1,v1,stdev,M,q)==0 and Reg1(z2,v2,stdev,M,q)==0:
                pi+=[c,z1,z2]
                #print("pi",len(pi))
                break
            # 如果没生成 pi，返回显式空
        return pi


    def CheckPi(prof,pku,resu,stdev,kappa,m,d,q,bfA,bfB,hx,Bc):
        c=prof[0]
        # print("c :",c)
        z1=prof[1]
        z2=prof[2]
        tA=pku[0]
        tAp=resu[0]
        tB=pku[1]
        tb=resu[1]
        normz1=VecInner(z1,z1,q)
        normz2=VecInner(z2,z2,q)
        rv=int(stdev**2*2*m*d+1)
        if normz1> rv or normz2>rv:
            print(f"Norm check failed: normz1={normz1}, normz2={normz2}, rv={rv}")
            return false
        Az1=MatrixVectorMul(bfA,z1,n,m,d,q)
        ctA=[c*i for i in tA]
        vecw1p=Vectorsub(Az1,ctA)
        # print(f"CheckPi - vecw1p[0]: {vecw1p[0]}")
        Az2=MatrixVectorMul(bfA,z2,n,m,d,q)
        ctAp=[c*i for i in tAp]
        vecw2p=Vectorsub(Az2,ctAp)
        Bz1=MatrixVectorMul(bfB,z1,kappa,m,d,q)
        vecvp1=Vectormul(Bz1,hx)
        vecvp2=z2[0]
        vecvp3=Vectormul(tB,hx)
        vecvp3f=vecvp3-tb
        vecvp=vecvp1-vecvp2-c*vecvp3f
        cp=HashC([vecw1p,vecw2p,[vecvp]],d,Bc)
        # print("cp:",cp)
        # 比较多项式的系数
        # c_coef = GetCoef(d, c)
        # cp_coef = GetCoef(d, cp)
        # for i in range(d):
        #     if c_coef[i] != cp_coef[i]:
        #         print(f"Coefficient mismatch at index {i}: c={c_coef[i]}, cp={cp_coef[i]}")
        if cp!=c:
            # print(cp)
            # print(c)
            print("Hash mismatch, rejecting proof")
            return false
        return true

    def Findq(logq):
        q=gmpy2.next_prime(2**logq)
        while q%8!=5:
            q=gmpy2.next_prime(q+2)
        return q
    def setparams(i):
        params=[[10, 36, 26, 11, 8, 5435817984.0*sqrt(7)*sqrt(6)*sqrt(2)], [21, 59, 28, 12, 9, (3.262849744896e+16)*sqrt(10)*sqrt(7)*sqrt(2)], [34, 86, 34, 14, 11, (3.8057879424466944e+24)*sqrt(14)*sqrt(7)*sqrt(2)], [49, 116, 40, 16, 13, (1.9799989970761797e+34)*sqrt(7)], [65, 148, 49, 18, 16, (1.29815281444998e+43)*sqrt(22)*sqrt(7)*sqrt(2)], [82, 182, 55, 21, 18, (1.7160552575395832e+53)*sqrt(26)*sqrt(7)*sqrt(2)], [99, 217, 64, 23, 21, (6.232512024172087e+63)*sqrt(30)*sqrt(7)*sqrt(2)], [118, 253, 72, 26, 24, (5.383062954803521e+74)*sqrt(34)*sqrt(7)*sqrt(2)], [136, 291, 79, 29, 26, (9.920304752532007e+85)*sqrt(38)*sqrt(7)*sqrt(2)], [155, 329, 88, 32, 29, (3.5849083657736404e+97)*sqrt(42)*sqrt(7)*sqrt(2)], [175, 369, 97, 35, 32, (2.374318877950044e+109)*sqrt(46)*sqrt(7)*sqrt(2)], [195, 409, 106, 38, 35, (2.7270444757372826e+122)*sqrt(7)], [215, 449, 115, 41, 38, (1.5561354924465855e+134)*sqrt(7)*sqrt(6)*sqrt(2)], [236, 491, 126, 44, 42, (1.5714894027927937e+146)*sqrt(58)*sqrt(7)*sqrt(2)], [257, 533, 136, 47, 45, (7.334003211056399e+158)*sqrt(62)*sqrt(7)*sqrt(2)], [279, 576, 145, 50, 48, (5.1220775117639e+171)*sqrt(66)*sqrt(7)*sqrt(2)], [300, 619, 154, 54, 51, (5.219541793839287e+184)*sqrt(70)*sqrt(7)*sqrt(2)], [322, 663, 163, 57, 54, (7.589309096222472e+197)*sqrt(74)*sqrt(7)*sqrt(2)], [344, 707, 175, 60, 58, (1.5436134763292818e+211)*sqrt(78)*sqrt(7)*sqrt(2)], [367, 752, 184, 64, 61, (4.314532843874707e+224)*sqrt(82)*sqrt(7)*sqrt(2)], [389, 797, 195, 67, 65, (1.630997586329492e+238)*sqrt(86)*sqrt(7)*sqrt(2)], [412, 842, 205, 70, 68, (2.465734131678997e+252)*sqrt(10)*sqrt(7)*sqrt(2)], [435, 888, 214, 74, 71, (5.449397988709671e+265)*sqrt(94)*sqrt(7)*sqrt(2)], [458, 935, 226, 77, 75, (6.575804086726498e+280)*sqrt(7)], [481, 981, 235, 81, 78, (5.2056351050921e+293)*sqrt(102)*sqrt(7)*sqrt(2)]]
        return [3*i+1,2*i+1,params[i-1][0],params[i-1][1],params[i-1][2],params[i-1][3],params[i-1][4]]
    ##dprf ring version
    para=setparams(num)
    N=para[0]
    T=para[1]
    m=para[4]
    n=para[5]
    kappa=para[6]
    s=2**para[2]*1.0##sc
    q=Findq(para[3])
    F=N-T
    d=128
    p=2
    beta=1024###结果正确的概率为1-1/correct_prob
    Bc=2
    ##用于零知识
    gamma=19
    eta=59
    sn=gamma * eta *math.sqrt(m)*s
    M=math.e**(14/gamma+1/(2*gamma**2))##Reg_1算法预期迭代次数
    N_Factor=int(gmpy2.fac(N))
    F_Factor=int(gmpy2.fac(F))
    ##模多项式环的表示
    R.<xx>=Zmod(q)[]
    f=R.cyclotomic_polynomial(2*d)
    S = R.quotient(f, 'x')
    x = S.gen()
    start_time = time.time()
    print(f"Running role: {ROLE}, user: {USER_ID}")
    print(f"Parameters: N={N}, T={T}, m={m}, n={n}, kappa={kappa}, s={s}, q={q}")

    # 根据N生成带参数的文件名
    setup_key_sharing_file = os.path.join(SHARE_DIR, f'setup_key_sharing_N{N}.pkl')
    veck_file = os.path.join(SHARE_DIR, f'veck_N{N}.pkl')
    param_file = os.path.join(SHARE_DIR, f'params_N{N}.pkl')
    
    # 检查文件是否存在并可读
    setup_exists = os.path.exists(setup_key_sharing_file)
    veck_exists = os.path.exists(veck_file)
    param_exists = os.path.exists(param_file)
    
    # 检查保存的参数是否与当前参数匹配
    params_match = False
    if setup_exists and veck_exists and param_exists:
        try:
            with open(param_file, 'rb') as f:
                saved_params = pickle.load(f)
            # 比较关键参数
            if saved_params['N'] == N and saved_params['q'] == q and saved_params['d'] == d:
                params_match = True
                print("Saved parameters match current configuration.")
            else:
                print(f"Saved parameters ({saved_params}) do not match current configuration (N={N}, q={q}, d={d}).")
        except Exception as e:
            print(f"Error loading parameters: {e}")
            params_match = False
    
    if setup_exists and veck_exists and params_match:
        try:
            # 加载保存的全局变量
            with open(setup_key_sharing_file, 'rb') as f:
                print(f"Loading {setup_key_sharing_file}")
                bfA, bfB, veckU, vecrU, pkU = pickle.load(f)
                print(f"Successfully loaded {setup_key_sharing_file}")
            
            with open(veck_file, 'rb') as f:
                print(f"Loading {veck_file}")
                veck = pickle.load(f)
                print(f"Successfully loaded {veck_file}")
            
            print("Loaded Setup, KeySharing variables and veck from file.")
        except Exception as e:
            print(f"Error loading files: {e}")
            # 如果加载失败，重新执行Setup和KeySharing
            setup_exists = False
            veck_exists = False
            params_match = False
    
    if not setup_exists or not veck_exists or not params_match:
        print("Regenerating parameters and data due to mismatch or missing files.")
        # 删除旧文件
        for file in [setup_key_sharing_file, veck_file, param_file]:
            if os.path.exists(file):
                try:
                    os.remove(file)
                    print(f"Deleted old file: {file}")
                except Exception as e:
                    print(f"Error deleting file {file}: {e}")
        
        ##Setup
        print("Executing Setup phase...")
        bfA = SampleMatrix(n, m, d, q)
        # print("bfA:",bfA)
        bfB = SampleMatrix(kappa, m, d, q)
        # print("bfB:",bfB)

        ##KeySharing
        print("Executing KeySharing phase...")
        veck = SampleMatrix(1, kappa, d, q)
        veck = veck[0]##矩阵表示改为向量表示
        
        # 保存 veck
        with open(veck_file, 'wb') as f:
            pickle.dump(veck, f)
            print(f"Saved veck to {veck_file}")
            
        veckU = Shareing(veck, kappa, d, N, T, q)
        vecrU = []
        pkU = []
        for i in range(N):
            vecru = SampleGas(d, m, s)##r1u
            vecrU += [vecru]
            vectAu = MatrixVectorMul(bfA, vecru, n, m, d, q)
            bfBvecru = MatrixVectorMul(bfB, vecru, kappa, m, d, q)
            vectBu = Vectoradd(bfBvecru, veckU[i])
            pkU += [[vectAu, vectBu]]

        # 保存全局变量
        with open(setup_key_sharing_file, 'wb') as f:
            pickle.dump((bfA, bfB, veckU, vecrU, pkU), f)
            print(f"Saved setup and key sharing data to {setup_key_sharing_file}")
        
        # 保存参数
        with open(param_file, 'wb') as f:
            pickle.dump({'N': N, 'q': q, 'd': d}, f)
            print(f"Saved parameters to {param_file}")

    piU=[]
    resU=[]
    vecx="a random string as input a"
    hx=Hashf(vecx,kappa,d,q)
    timezk=0
    if ROLE == 'worker':
        start_peval = time.time()
        print(f"User {USER_ID} starting PEval")
        # 仅处理一个用户的 PEval 计算
        # vecx="a random string as input a"
        # hx = Hashf(vecx, kappa, d, q)
        vecrpu = SampleGas(d, m, s)  # r2u
        brpu = vecrpu[0]
        hxku = Vectormul(hx, veckU[USER_ID - 1])
        tbu = brpu + hxku
        vectpAu = MatrixVectorMul(bfA, vecrpu, n, m, d, q)
        # 生成零知识证明
        zk_time_1 = time.time()
        # piU += [GenPi(bfA, bfB, hx, vecrU[i], vecrpu, kappa, m, d, sn, s, Bc)]
        piU = [GenPi(bfA, bfB, hx, vecrU[USER_ID - 1], vecrpu, kappa, m, d, sn, s, Bc)]
        # print("piU：",piU)
        # piU.append(tuple(pi_result))
        zk_time_2 = time.time()
        print("timezk",zk_time_2-zk_time_1)
        timezk+=zk_time_2-zk_time_1
        # resU.append([vectpAu, tbu])  # tbu 是一个多项式
        resU = [vectpAu, tbu]
        # print("resU:",resU)
        # 将 resU 和 piU 转换为 numpy 数组并保存
        try:
            # 保存 piU 和 resU 为原始 list，不转 np.array
            with open(os.path.join(SHARE_DIR, f'resU_user{USER_ID}.pkl'), 'wb') as f:
                pickle.dump(resU, f)
            with open(os.path.join(SHARE_DIR, f'piU_user{USER_ID}.pkl'), 'wb') as f:
                pickle.dump(piU, f)          # 当前的 pi_result 是一个 tuple：(c, z1, z2)
            with open(os.path.join(SHARE_DIR, f'pkU_user{USER_ID}.pkl'), 'wb') as f:
                pickle.dump(pkU[USER_ID - 1], f)
            # hx 是 numpy 数组，可以直接保存
            np.save(os.path.join(SHARE_DIR, f'hx.npy'), hx) if USER_ID == 1 else None
        except Exception as e:
            print(f"Error saving resU or piU: {e}")

        end_peval = time.time()
        print("PEval:",end_peval-start_peval)
    # print("当前 share 目录下的内容为：", os.listdir(SHARE_DIR))

    ##FinalEval
    if ROLE == 'final':    
    # FinalEval 阶段，只需由主节点执行
        # 从共享目录中读取 resU/piU
        timevzk = 0
        start_finaleval = time.time()
        Vuse=[]
        for i in range(1, T + 1):  
            try:
                with open(os.path.join(SHARE_DIR, f'resU_user{i}.pkl'), 'rb') as f:
                    resU_i = pickle.load(f)
                with open(os.path.join(SHARE_DIR, f'piU_user{i}.pkl'), 'rb') as f:
                    piU_i = pickle.load(f)
                    if isinstance(piU_i, (list, tuple)) and len(piU_i) == 1 and isinstance(piU_i[0], (list, tuple)):
                        piU_i = piU_i[0]
                with open(os.path.join(SHARE_DIR, f'pkU_user{i}.pkl'), 'rb') as f:
                    pkU_i = pickle.load(f)
            except Exception as e:
                print(f"Missing file for user {i}: {e}")
                continue

            try:
                hx = np.load(os.path.join(SHARE_DIR, 'hx.npy'), allow_pickle=True).tolist()
                if CheckPi(piU_i, pkU_i, resU_i, sn, kappa, m, d, q, bfA, bfB, hx, Bc):
                    Vuse.append(i)
                    pkU.append(pkU_i)
                    resU.append(resU_i)
                    piU.append(piU_i)
                else:
                    print(f"User {i} failed ZKP check")
            except Exception as e:
                print(f"Error verifying user {i}: {e}")
        v_end = time.time()
        timevzk = v_end - start_finaleval
        print("timevzk:",timevzk)
        if len(Vuse) < T:
            print(f"Not enough valid users ({len(Vuse)}) to reach threshold {T}. Aborting.")
            return
        
        # 检查 resU[j] 的长度是否足够
        zi1_c = []
        for j in Vuse:
            if len(resU[j-1]) >= 2:
                zi1_c.append(GetCoef(d, resU[j-1][1]))
            else:
                print(f"resU[{j-1}] has insufficient length. Skipping this user.")

        if len(zi1_c) < len(Vuse):
            print("Not all users have valid resU data. Aborting.")
            return
        
        # zi1_c = [GetCoef(d, resU[j][1]) for j in Vuse]
        z1_c = [recover_secret_special(
                    [zi1_c[j][i] for j in range(len(Vuse))],
                    [j+1 for j in range(len(Vuse))],  # Lagrange 插值中的 x 值（用户编号）
                    N_Factor, 1, q)
                for i in range(d)]
        z1 = SamplePolyCo(d, z1_c)
        barg0 = GetCoef(d, z1)[0]
        feres = int(barg0 / (q / p))
        print("feres:",feres)
        end_finaleval = time.time()
        print("FinalEval:",end_finaleval-start_finaleval)

        ##Eval
        # start_eval = time.time()
        with open(veck_file, 'rb') as f:
            veck = pickle.load(f)
        hxk=Vectormul(hx,veck)
        gpc=GetCoef(d,hxk)
        tmpf=N_Factor*gpc[0]%q
        #print(tmpf)##这里不涉及高斯噪声，所以不用计算mymod
        bargp0=int(tmpf/(q/p))
        print("bargp0",bargp0)

print(f"Running role: {ROLE}, user: {USER_ID}")
alldct(NUM_I, mode=os.getenv('ROLE'), user_index=USER_ID - 1)


##4  0.014027595520019531 2.6793527603149414 25.18831443786621 3.887711763381958 6.910128116607666
##7  0.014235496520996094 4.298682689666748 42.61752247810364 7.853028297424316 8.356310606002808
##10 0.026088714599609375 13.319802522659302 146.58300113677979 22.358633995056152 20.021886587142944
##13 0.026048660278320312 20.96372675895691 271.92235374450684 34.700652837753296 27.18429708480835
##16 0.028507232666015625 32.84743928909302 490.17990159988403 52.862579107284546 39.45332169532776
##19 0.030170440673828125 48.30398511886597 540.2786636352539 77.76343011856079 51.078370094299316
##22 0.032071590423583984 68.2996883392334 753.3100168704987 107.30013537406921 68.08728075027466
##25 0.03447079658508301 97.41797924041748 819.1081779003143 148.80373549461365 87.47875475883484
##28 0.03502154350280762 130.8589527606964 1470.480384349823 197.34385752677917 106.10798192024231
##31 0.0381162166595459 179.67422366142273 1939.414316892624 253.27185654640198 133.17090940475464
##34 0.04021739959716797 224.9040744304657 2651.0636401176453 324.65426325798035 160.98098826408386  此时就需要5分钟了

2
2
4 0.007107734680175781 1.5449440479278564 14.204541444778442 2.4985029697418213 3.289034128189087 13.270437717437744 1.2283971309661865
1
1
7 0.007254600524902344 2.902761936187744 35.02891755104065 4.597910165786743 3.979844093322754 33.33390951156616 2.3038482666015625
1
1
10 0.02056121826171875 12.349298238754272 192.85302686691284 21.36642551422119 14.083102464675903 185.26534461975098 11.243284225463867
1
1
13 0.02230215072631836 20.75442147254944 350.49741530418396 33.71887946128845 19.37418270111084 338.9733006954193 18.883507013320923
2
2
16 0.02466559410095215 33.41847348213196 600.3568091392517 57.15347671508789 28.194189310073853 582.6625289916992 30.70600914955139
1
1
19 0.02603626251220703 49.06621956825256 508.7485656738281 79.43377113342285 36.561233043670654 485.0429835319519 45.421026945114136


IndexError: list index out of range

In [1]:
import os
import pika

rabbitmq_host = os.getenv("RABBITMQ_HOST", "localhost")
print(f"Connecting to RabbitMQ host: {rabbitmq_host}")

try:
    connection = pika.BlockingConnection(pika.ConnectionParameters(host=rabbitmq_host))
    channel = connection.channel()
    print("Connected to RabbitMQ successfully!")
except Exception as e:
    print(f"Failed to connect RabbitMQ: {e}")


Connecting to RabbitMQ host: localhost
Connected to RabbitMQ successfully!


In [39]:
import os
import socket

FINAL_HOST = os.getenv('FINAL_HOST', 'finaleval_1')
FINAL_PORT = int(os.getenv('FINAL_PORT', 5000))

print(f"Connecting to {FINAL_HOST}:{FINAL_PORT}")
try:
    ip = socket.gethostbyname(FINAL_HOST)
    print(f"Resolved {FINAL_HOST} to {ip}")
except Exception as e:
    print(f"DNS resolution error: {e}")

try:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((FINAL_HOST, FINAL_PORT))
        print("Connected successfully")
except Exception as e:
    print(f"Connection failed: {e}")


Connecting to finaleval_1:5000
DNS resolution error: [Errno -3] Temporary failure in name resolution
Connection failed: [Errno -3] Temporary failure in name resolution


In [40]:
import os
ROLE = os.getenv('ROLE', 'none')
print(f"Starting with ROLE={ROLE}")
if ROLE == 'worker':
    print("I am a worker, doing worker stuff")
elif ROLE == 'final':
    print("I am finaleval, doing finaleval stuff")
else:
    print("Unknown role")


Starting with ROLE=none
Unknown role


In [3]:
##ACNS 2024
##LISSS Section 2.4
import numpy as np
import gmpy2
import hashlib
import time
import ntl
from itertools import combinations
def allACNS(num):
    def MAnd(A, B):
        if A.shape[1] == 1 and B.shape[1] == 1:
            ca = A[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            cb = B[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            zero_B = np.zeros((B.shape[0], 1))  # 与cb同高度的零向量
            col1 = np.vstack((ca, zero_B))
            col2 = np.vstack((ca, cb)) 
            C = np.hstack((col1, col2))
            return C
        elif B.shape[1] == 1:
            ca = A[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            LA = A[:, 1:]                 # 剩余列
            cb = B[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            zero_B = np.zeros((B.shape[0], 1))  # 与cb同高度的零向量
            col1 = np.vstack((ca, zero_B))      # 第一列：ca和0
            col2 = np.vstack((ca, cb))          # 第二列：ca和cb
            col3 = np.vstack((LA, np.zeros((B.shape[0], LA.shape[1]))))  # 第三列：LA和0
            C = np.hstack((col1, col2, col3))
            return C
        else:
            # 提取矩阵A的第一列和剩余列
            ca = A[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            LA = A[:, 1:]                 # 剩余列

            # 提取矩阵B的第一列和剩余列
            cb = B[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            LB = B[:, 1:]                 # 剩余列

            # 创建零向量
            zero_A = np.zeros((A.shape[0], 1))  # 与ca同高度的零向量
            zero_B = np.zeros((B.shape[0], 1))  # 与cb同高度的零向量

            # 构建矩阵C的每一列
            col1 = np.vstack((ca, zero_B))      # 第一列：ca和0
            col2 = np.vstack((ca, cb))          # 第二列：ca和cb
            col3 = np.vstack((LA, np.zeros((B.shape[0], LA.shape[1]))))  # 第三列：LA和0
            col4 = np.vstack((np.zeros((A.shape[0], LB.shape[1])), LB))  # 第四列：0和LB

            # 合并所有列形成矩阵C
            C = np.hstack((col1, col2, col3, col4))

            return C
    def MOr(A, B):
        if A.shape[1] == 1:
            ca = A[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            cb = B[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            col1 = np.vstack((ca, cb))
            return col1
        else:
            # 提取矩阵A的第一列和剩余列
            ca = A[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            LA = A[:, 1:]                # 剩余列

            # 提取矩阵B的第一列和剩余列
            cb = B[:, 0].reshape(-1, 1)  # 第一列，reshape为列向量
            LB = B[:, 1:]                # 剩余列

            # 创建零向量
            zero_A = np.zeros((A.shape[0], LB.shape[1]))  # 与LA列数相同的零矩阵，行数与A相同
            zero_B = np.zeros((B.shape[0], LA.shape[1]))  # 与LA列数相同的零矩阵，行数与B相同

            # 构建矩阵C的每一列
            col1 = np.vstack((ca, cb))          # 第一列：ca和cb
            col2 = np.vstack((LA, zero_B))      # 第二列：LA和0
            col4 = np.vstack((zero_A, LB))      # 第四列：0和LB

            # 合并所有列形成矩阵C
            C = np.hstack((col1, col2, col4))

            return C
    def M_Creat(N,K):
        ##有K单位阵需要MAnd
        ##有C(N,K)个矩阵需要Mor
        A=np.array([[1]])
        B=np.array([[1]])
        for i in range(K-1):
            A=MAnd(A,B)
            #print(A)
        B=A
        # 使用NTL计算阶乘
        # N_Factor = ntl.factorial(N)
        # K_Factor = ntl.factorial(K)
        # F_Factor = ntl.factorial(N - K)
        # 原版计算逻辑
        N_Factor=int(gmpy2.fac(N))
        K_Factor=int(gmpy2.fac(K))
        F_Factor=int(gmpy2.fac(N-K))
        counter=N_Factor//K_Factor//F_Factor-1
        for i in range(counter):
            A=MOr(A,B)
            #print(A)
        return A
    def rho_Creat(e,n,veck,q):
        ##e是M矩阵的列数M.shape[1]，n是格的维度Z_q^n，veck是n维要分享的秘密
        rho=np.random.randint(0, q, size=(e, n))
        rho[0, :] = veck
        return rho
    def Sharing(M, N, K):
        """
        将矩阵 M 按每 K 行分组，并与集合 S1, S2, ..., SI 建立对应关系。

        参数:
        M (numpy.ndarray): 输入矩阵，形状为 (I * K, T)。
        N (int): 实体总数。
        K (int): 每个集合的大小。

        返回:
        dict: 每个实体对应的矩阵。
        """
        # 计算集合总数 I = C(N, K)
        I = len(list(combinations(range(N), K)))

        # 检查矩阵行数是否匹配
        if M.shape[0] != I * K:
            raise ValueError(f"矩阵行数应为 {I * K}，但实际为 {M.shape[0]}。")

        # 生成所有集合 S1, S2, ..., SI
        entities = range(N)
        sets = list(combinations(entities, K))

        # 初始化每个实体的矩阵
        entity_matrices = {i: [] for i in range(N)}
        entity_index={sets[i]: [] for i in range(I)}
        # 将矩阵 M 按每 K 行分组
        for i in range(I):
            # 获取当前子矩阵
            sub_matrix = M[i * K : (i + 1) * K, :]

            # 获取当前集合
            current_set = sets[i]
            entity_index[current_set].append(i)
            # 将子矩阵的每一行分配给集合中的实体
            for row_idx, entity in enumerate(current_set):
                entity_matrices[entity].append(list(sub_matrix[row_idx, :])+[i])


        # 将每个实体的列表转换为矩阵
        for entity in entity_matrices:
            entity_matrices[entity] = np.vstack(entity_matrices[entity])

        return entity_index,entity_matrices
    def find_set_index(sets, input_entities):

        ###input_entities (list): 输入的 K 个实体，例如 [Pa1, Pa2, ..., PaK]
        # 将输入的实体排序
        input_entities_sorted = sorted(input_entities)

        # 查找匹配的集合编号
        for i, s in enumerate(sets):
            if tuple(input_entities_sorted) == s:
                return i  # 返回集合编号

        return -1  # 如果未找到
    def Hashf(stri,q,n):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(stri.encode('utf-8'))
        res=[]
        for i in range(n):
            index=int(i)
            sha256_hash.update(index.to_bytes((index.bit_length() + 7) // 8, byteorder='big'))
            hash_hex=sha256_hash.hexdigest()
            val=int(hash_hex[:20], 16)
            res+=[val%q]
        return res

    def codesave():##多余的代码
        row=sharings[1].shape[0]
        col=sharings[1].shape[1]
        res=np.array([0.0]*n)
        for j in range(row):
            print(int(sharings[V[0]][j][col-1]))
            if int(sharings[V[0]][j][col-1])==ind:
                res+=np.array(sharings[V[0]][j][:col-1])
                break
        print("res 1",res)
        for ent in V[1:]:
            for j in range(row):
                if sharings[ent][j][col-1]==ind:
                    res-=np.array(sharings[ent][j][:col-1])
        print(res)
        
        #print(sets_ind)
        #for entity, matrix in sharings.items():
        #    print(f"\n实体 {entity} 的矩阵：")
        #    print(matrix)
        


    N=3*num+1
    K=2*num+1
    n=412
    q=2**32
    q1=2**28
    p=2
    print("N:",N)
    ###Sharing
    start_time=time.time()
    M=M_Creat(N,K)
    #print(M)
    veck=np.random.randint(0, q, size=(1, n))
    #print(veck)
    rho=rho_Creat(M.shape[1],n,veck,q)
    #print(rho)
    share_matrix=np.matmul(M, rho)
    #print("share_matrix",share_matrix)
    sets_ind,sharings=Sharing(share_matrix, N, K)
    end_time1=time.time()
    row=sharings[1].shape[0]
    col=sharings[1].shape[1]
    
    ###PartialEval
    vecx="a random string as input a"
    hx=Hashf(vecx,q,n)
    pvalall=[]
    for ent in range(N):
        pvalent=[]
        for ro in range(row):
            rowval=[int(sharings[ent][ro][col-1])]
            key=sharings[ent][ro][:col-1]
            hxkey=np.dot(hx,key)
            rowval+=[np.round(hxkey*q1/q).astype(int) % q1]
            pvalent+=[rowval]
        pvalall+=[pvalent]
    #print("pvalall",pvalall)
    end_time2=time.time()
    print("PEval:",end_time2-end_time1)
    ###FinalEval
    V=[i for i in range(K)]
    V=tuple(V)
    ind=sets_ind[V][0]
    #print("ind",ind)
    res=0
    for rows in pvalall[V[0]]:##确保V[0]是V集合中最小的
        #print("rows",rows)
        if rows[0]==ind:
            res=rows[1:][0]
            break
    for i in range(1,K):
        for rows in pvalall[V[i]]:##确保V[0]是V集合中最小的
            if rows[0]==ind:
                res-=rows[1:][0]
                break
    final=np.round(res*p/q1).astype(int) % p
    end_time3=time.time()
    print("FinalEval:",end_time3-end_time2)
    ###Eval
    cor=np.round(np.dot(hx,veck[0])*p/q).astype(int) % p
    # print("final",final)
    # print("cor",cor)
    # print(end_time3-end_time2,end_time2-end_time1,end_time1-start_time)

for i in range(1,12):
    allACNS(i)

N: 4
PEval: 0.03509664535522461
FinalEval: 0.00019669532775878906
N: 7
PEval: 0.26528406143188477
FinalEval: 0.0001823902130126953
N: 10
PEval: 2.1796746253967285
FinalEval: 0.0008113384246826172
N: 13
PEval: 16.17936658859253
FinalEval: 0.0007145404815673828
N: 16


KeyboardInterrupt: 